## Classification with Logistic Regression

## Agenda

- Classification and its types
- **Problem statement**
- **Solving classification problems with linear regression**
- Building blocks of Logistic Regression
    - **Sigmoid**
    - Odds ratio
    - **Decision boundary stability**
- Cost function intuition
- Model building with scikit-learn
- Evaluation metrics

## What is Classification?

- Group data according to some criteria based on attributes. Ex:
    - Predict genre of movie as **horror/suspense/thriller** based on plot, duration, actors etc.  
    - Classify emails as **spam/ham email** based on email subject, email message, attachments etc.
- `Different from regression`: Predict categories (discrete value) not continuous values. 
- Supervised learning mode; unsupervised version similar to classification also exists (clustering).

- Supervised learning method <img src='../images/supervised.png' wodth=500>

- **`Unsupervised`**: We don't know about the targets of data points and goal is to categorize data into a finite number of categories based on given features
- **`Supervised`**: Information about true targets are known (shown by colored points) and goal is predict that information (number/category) on unseen data

## Problem Statement

